Historical sales data,from  2010-02-05(2010Feb) to around 2012-11-01(2012Nov). 

In [1]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
#from pickle import dump
#from sklearn import __version__ as sklearn_version
#from sklearn.decomposition import PCA
from sklearn.preprocessing import scale
from sklearn.model_selection import train_test_split, cross_validate, GridSearchCV, learning_curve
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.dummy import DummyRegressor
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.pipeline import make_pipeline
from sklearn.impute import SimpleImputer
from sklearn.feature_selection import SelectKBest, f_regression
import datetime

In [2]:
#%load_ext autoreload
#%autoreload 2

#### Load the data

In [3]:
retail_data = pd.read_csv('/Users/jisasv1/Documents/dataScience/Datasets/Capstone2Data/retail_data_eda.csv')

In [4]:
retail_data.shape

(421570, 22)

In [5]:
retail_data.head()

,Unnamed: 0,Store,Dept,Date,Weekly_Sales,IsHoliday_x,Type,Size,Temperature,Fuel_Price,...,MarkDown3,MarkDown4,MarkDown5,CPI,Unemployment,IsHoliday_y,Month,AvgWeeklySalesperStore,AvgWeeklySalesperDept,AverageMonthlySales
0,0,1,1,2010-02-05,24924.50,False,A,151315,42.31,2.572,...,0.0,0.0,0.0,211.096358,8.106,False,2,21710.543621,19213.485088,16008.779217
1,1,1,2,2010-02-05,50605.27,False,A,151315,42.31,2.572,...,0.0,0.0,0.0,211.096358,8.106,False,2,21710.543621,43607.020113,16008.779217
2,2,1,3,2010-02-05,13740.12,False,A,151315,42.31,2.572,...,0.0,0.0,0.0,211.096358,8.106,False,2,21710.543621,11793.698516,16008.779217
3,3,1,4,2010-02-05,39954.04,False,A,151315,42.31,2.572,...,0.0,0.0,0.0,211.096358,8.106,False,2,21710.543621,25974.630238,16008.779217
4,4,1,5,2010-02-05,32229.38,False,A,151315,42.31,2.572,...,0.0,0.0,0.0,211.096358,8.106,False,2,21710.543621,21365.583515,16008.779217


In [6]:
retail_data.drop(['Unnamed: 0'], axis=1, inplace=True) # pls specify inplace=True to do operation inplace.

In [7]:
retail_data.head()

,Store,Dept,Date,Weekly_Sales,IsHoliday_x,Type,Size,Temperature,Fuel_Price,MarkDown1,...,MarkDown3,MarkDown4,MarkDown5,CPI,Unemployment,IsHoliday_y,Month,AvgWeeklySalesperStore,AvgWeeklySalesperDept,AverageMonthlySales
0,1,1,2010-02-05,24924.50,False,A,151315,42.31,2.572,0.0,...,0.0,0.0,0.0,211.096358,8.106,False,2,21710.543621,19213.485088,16008.779217
1,1,2,2010-02-05,50605.27,False,A,151315,42.31,2.572,0.0,...,0.0,0.0,0.0,211.096358,8.106,False,2,21710.543621,43607.020113,16008.779217
2,1,3,2010-02-05,13740.12,False,A,151315,42.31,2.572,0.0,...,0.0,0.0,0.0,211.096358,8.106,False,2,21710.543621,11793.698516,16008.779217
3,1,4,2010-02-05,39954.04,False,A,151315,42.31,2.572,0.0,...,0.0,0.0,0.0,211.096358,8.106,False,2,21710.543621,25974.630238,16008.779217
4,1,5,2010-02-05,32229.38,False,A,151315,42.31,2.572,0.0,...,0.0,0.0,0.0,211.096358,8.106,False,2,21710.543621,21365.583515,16008.779217


#### Train/Test/Split

In [8]:
#Check data types of retail_data 
retail_data.dtypes

Store                       int64
Dept                        int64
Date                       object
Weekly_Sales              float64
IsHoliday_x                  bool
Type                       object
Size                        int64
Temperature               float64
Fuel_Price                float64
MarkDown1                 float64
MarkDown2                 float64
MarkDown3                 float64
MarkDown4                 float64
MarkDown5                 float64
CPI                       float64
Unemployment              float64
IsHoliday_y                  bool
Month                       int64
AvgWeeklySalesperStore    float64
AvgWeeklySalesperDept     float64
AverageMonthlySales       float64
dtype: object

Seperate categorical and numerical columns

In [9]:
#Let's do our feature selection.
#Make a variable called 'categorical', and a list containing all Categorical(object) types columns 
categorical = ['Store','Type','Dept','IsHoliday_x','IsHoliday_y']

In [10]:
X_cat= retail_data[categorical]

##### Create Dummies for categorical columns

In [11]:
#Creating dummies for the features columns
X_cat= pd.get_dummies(X_cat)   
X_cat

,Store,Dept,IsHoliday_x,IsHoliday_y,Type_A,Type_B,Type_C
0,1,1,False,False,1,0,0
1,1,2,False,False,1,0,0
2,1,3,False,False,1,0,0
3,1,4,False,False,1,0,0
4,1,5,False,False,1,0,0
...,...,...,...,...,...,...,...
421565,45,93,False,False,0,1,0
421566,45,94,False,False,0,1,0
421567,45,95,False,False,0,1,0
421568,45,97,False,False,0,1,0


In [12]:
# Make a dependent variable called y, and assign it the column which we have to predict
y= retail_data.AvgWeeklySalesperDept

In [13]:
#Make a variable called numerical and a list containing all Numerical columns
X_numerical= retail_data.drop(['Date','Store','Type','Dept','AvgWeeklySalesperDept','IsHoliday_x','IsHoliday_y'], axis=1)
X_numerical

,Weekly_Sales,Size,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,CPI,Unemployment,Month,AvgWeeklySalesperStore,AverageMonthlySales
0,24924.50,151315,42.31,2.572,0.00,0.00,0.0,0.00,0.00,211.096358,8.106,2,21710.543621,16008.779217
1,50605.27,151315,42.31,2.572,0.00,0.00,0.0,0.00,0.00,211.096358,8.106,2,21710.543621,16008.779217
2,13740.12,151315,42.31,2.572,0.00,0.00,0.0,0.00,0.00,211.096358,8.106,2,21710.543621,16008.779217
3,39954.04,151315,42.31,2.572,0.00,0.00,0.0,0.00,0.00,211.096358,8.106,2,21710.543621,16008.779217
4,32229.38,151315,42.31,2.572,0.00,0.00,0.0,0.00,0.00,211.096358,8.106,2,21710.543621,16008.779217
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
421565,2487.80,118221,58.85,3.882,4018.91,58.08,100.0,211.94,858.33,192.308899,8.667,10,11662.897315,15243.855576
421566,5203.31,118221,58.85,3.882,4018.91,58.08,100.0,211.94,858.33,192.308899,8.667,10,11662.897315,15243.855576
421567,56017.47,118221,58.85,3.882,4018.91,58.08,100.0,211.94,858.33,192.308899,8.667,10,11662.897315,15243.855576
421568,6817.48,118221,58.85,3.882,4018.91,58.08,100.0,211.94,858.33,192.308899,8.667,10,11662.897315,15243.855576


##### Scale the numerical columns to prep for model creation

In [14]:
#scale data
from sklearn import preprocessing
import numpy as np
#build scaler based on training data and apply it to test data to then also scale the test data
scaler = preprocessing.StandardScaler().fit(X_numerical)
X_numerical_scaled=scaler.transform(X_numerical)
#X_test_scaled=scaler.transform(X_test)

##### Time-Series split

In [15]:
#from sklearn.model_selection import TimeSeriesSplit
#tscv = TimeSeriesSplit(n_splits=2, max_train_size=None, test_size=None, gap=0)
#X_test, y_train, y_test
#for train_index, test_index in tscv.split(X_merged):
#train_index, test_index = tscv.split(X_merged)
    #X_train, X_test = X_merged.loc[train_index], X_merged.loc[test_index]
    #y_train, y_test = y[train_index], y[test_index]
                                                             #use .loc for X_merged since it's a df

Refer :https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.TimeSeriesSplit.html
ref:https://stackoverflow.com/questions/45352909/pandas-indexingerror-unalignable-boolean-series-provided-as-indexer 

In [16]:
print(X_numerical_scaled)
#print(X_train_scaled)

[[ 0.39378188  0.23920895 -0.96379804 ... -1.37194493  0.82630205
   0.02412105]
 [ 1.52453759  0.23920895 -0.96379804 ... -1.37194493  0.82630205
   0.02412105]
 [-0.09868005  0.23920895 -0.96379804 ... -1.37194493  0.82630205
   0.02412105]
 ...
 [ 1.76284337 -0.30348902 -0.06721947 ...  1.09474444 -0.62281246
  -0.6463014 ]
 [-0.40349236 -0.30348902 -0.06721947 ...  1.09474444 -0.62281246
  -0.6463014 ]
 [-0.65626152 -0.30348902 -0.06721947 ...  1.09474444 -0.62281246
  -0.6463014 ]]


In [17]:
type(X_cat)

pandas.core.frame.DataFrame

In [18]:
X_numerical_scaled = pd.DataFrame(X_numerical_scaled)
X_numerical_scaled

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,0.393782,0.239209,-0.963798,-1.720834,-0.427943,-0.173069,-0.084662,-0.278117,-0.395181,1.018774,0.078201,-1.371945,0.826302,0.024121
1,1.524538,0.239209,-0.963798,-1.720834,-0.427943,-0.173069,-0.084662,-0.278117,-0.395181,1.018774,0.078201,-1.371945,0.826302,0.024121
2,-0.098680,0.239209,-0.963798,-1.720834,-0.427943,-0.173069,-0.084662,-0.278117,-0.395181,1.018774,0.078201,-1.371945,0.826302,0.024121
3,1.055551,0.239209,-0.963798,-1.720834,-0.427943,-0.173069,-0.084662,-0.278117,-0.395181,1.018774,0.078201,-1.371945,0.826302,0.024121
4,0.715425,0.239209,-0.963798,-1.720834,-0.427943,-0.173069,-0.084662,-0.278117,-0.395181,1.018774,0.078201,-1.371945,0.826302,0.024121
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
421565,-0.594133,-0.303489,-0.067219,1.136222,0.236078,-0.161646,-0.066576,-0.223697,-0.191187,0.539003,0.379281,1.094744,-0.622812,-0.646301
421566,-0.474566,-0.303489,-0.067219,1.136222,0.236078,-0.161646,-0.066576,-0.223697,-0.191187,0.539003,0.379281,1.094744,-0.622812,-0.646301
421567,1.762843,-0.303489,-0.067219,1.136222,0.236078,-0.161646,-0.066576,-0.223697,-0.191187,0.539003,0.379281,1.094744,-0.622812,-0.646301
421568,-0.403492,-0.303489,-0.067219,1.136222,0.236078,-0.161646,-0.066576,-0.223697,-0.191187,0.539003,0.379281,1.094744,-0.622812,-0.646301


In [19]:
#Reassign the solumn names
X_numerical_scaled.columns= X_numerical.columns

In [20]:
X_numerical_scaled

,Weekly_Sales,Size,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,CPI,Unemployment,Month,AvgWeeklySalesperStore,AverageMonthlySales
0,0.393782,0.239209,-0.963798,-1.720834,-0.427943,-0.173069,-0.084662,-0.278117,-0.395181,1.018774,0.078201,-1.371945,0.826302,0.024121
1,1.524538,0.239209,-0.963798,-1.720834,-0.427943,-0.173069,-0.084662,-0.278117,-0.395181,1.018774,0.078201,-1.371945,0.826302,0.024121
2,-0.098680,0.239209,-0.963798,-1.720834,-0.427943,-0.173069,-0.084662,-0.278117,-0.395181,1.018774,0.078201,-1.371945,0.826302,0.024121
3,1.055551,0.239209,-0.963798,-1.720834,-0.427943,-0.173069,-0.084662,-0.278117,-0.395181,1.018774,0.078201,-1.371945,0.826302,0.024121
4,0.715425,0.239209,-0.963798,-1.720834,-0.427943,-0.173069,-0.084662,-0.278117,-0.395181,1.018774,0.078201,-1.371945,0.826302,0.024121
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
421565,-0.594133,-0.303489,-0.067219,1.136222,0.236078,-0.161646,-0.066576,-0.223697,-0.191187,0.539003,0.379281,1.094744,-0.622812,-0.646301
421566,-0.474566,-0.303489,-0.067219,1.136222,0.236078,-0.161646,-0.066576,-0.223697,-0.191187,0.539003,0.379281,1.094744,-0.622812,-0.646301
421567,1.762843,-0.303489,-0.067219,1.136222,0.236078,-0.161646,-0.066576,-0.223697,-0.191187,0.539003,0.379281,1.094744,-0.622812,-0.646301
421568,-0.403492,-0.303489,-0.067219,1.136222,0.236078,-0.161646,-0.066576,-0.223697,-0.191187,0.539003,0.379281,1.094744,-0.622812,-0.646301


In [21]:
#Apply scaler Transform  to training and test set and also add y and DATE to it
#data = pd.concat([retail_data[['Weekly_Sales']].reset_index(drop=True), #reset index here
 #                   pd.DataFrame(scaler.transform(X[X_columns]),columns=X_columns)],axis=1)

Now Recombine the categorical columns,numerical columns and the dependant varuable y and Date column into a single dataframe.

In [22]:
data = pd.concat([X_numerical_scaled, X_cat,retail_data[['AvgWeeklySalesperDept','Date']]],axis=1)

In [23]:
data.head(10)

,Weekly_Sales,Size,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,CPI,...,AverageMonthlySales,Store,Dept,IsHoliday_x,IsHoliday_y,Type_A,Type_B,Type_C,AvgWeeklySalesperDept,Date
0,0.393782,0.239209,-0.963798,-1.720834,-0.427943,-0.173069,-0.084662,-0.278117,-0.395181,1.018774,...,0.024121,1,1,False,False,1,0,0,19213.485088,2010-02-05
1,1.524538,0.239209,-0.963798,-1.720834,-0.427943,-0.173069,-0.084662,-0.278117,-0.395181,1.018774,...,0.024121,1,2,False,False,1,0,0,43607.020113,2010-02-05
2,-0.098680,0.239209,-0.963798,-1.720834,-0.427943,-0.173069,-0.084662,-0.278117,-0.395181,1.018774,...,0.024121,1,3,False,False,1,0,0,11793.698516,2010-02-05
3,1.055551,0.239209,-0.963798,-1.720834,-0.427943,-0.173069,-0.084662,-0.278117,-0.395181,1.018774,...,0.024121,1,4,False,False,1,0,0,25974.630238,2010-02-05
4,0.715425,0.239209,-0.963798,-1.720834,-0.427943,-0.173069,-0.084662,-0.278117,-0.395181,1.018774,...,0.024121,1,5,False,False,1,0,0,21365.583515,2010-02-05
5,-0.450538,0.239209,-0.963798,-1.720834,-0.427943,-0.173069,-0.084662,-0.278117,-0.395181,1.018774,...,0.024121,1,6,False,False,1,0,0,4747.856188,2010-02-05
6,0.224683,0.239209,-0.963798,-1.720834,-0.427943,-0.173069,-0.084662,-0.278117,-0.395181,1.018774,...,0.024121,1,7,False,False,1,0,0,24161.237413,2010-02-05
7,1.063255,0.239209,-0.963798,-1.720834,-0.427943,-0.173069,-0.084662,-0.278117,-0.395181,1.018774,...,0.024121,1,8,False,False,1,0,0,30191.263517,2010-02-05
8,0.041818,0.239209,-0.963798,-1.720834,-0.427943,-0.173069,-0.084662,-0.278117,-0.395181,1.018774,...,0.024121,1,9,False,False,1,0,0,20206.681878,2010-02-05
9,0.649031,0.239209,-0.963798,-1.720834,-0.427943,-0.173069,-0.084662,-0.278117,-0.395181,1.018774,...,0.024121,1,10,False,False,1,0,0,18321.272351,2010-02-05


##### Save files 

#Now that we have our pre-processed and trained data,  let's save this data for modeling. 

In [24]:
data.to_csv('/Users/jisasv1/Documents/dataScience/Datasets/Capstone2Data/trained.csv')

The pre-processed data is saved and can be used for modeling.

In [25]:
#retail_data.to_csv('/Users/jisasv1/Documents/dataScience/Datasets/Capstone2Data/retail_data_Pre-processed.csv')

In [26]:
#Time_Series_data.to_csv('/Users/jisasv1/Documents/dataScience/Datasets/Capstone2Data/retail_data_timeS.csv')

In [27]:
#Let's now save the scaler,(the scaled values) to a file.
#import joblib 
#joblib.dump(model_res, '/Users/jisasv1/Documents/dataScience/Datasets/Capstone2Data/model.pkl') 
#dump(scaler, open('scaler.pkl', 'wb'))
#Refer : https://stackoverflow.com/questions/41993565/save-minmaxscaler-model-in-sklearn